# An Introduction to Dask

Dask is a flexible parallel computing library for Python. It allows dynamic scheduling of tasks for optimising computations and includes collections (extensions of arrays, lists and dataframes) for use with parallel data processing.

## Terminology

### General

#### Thread
A thread is the simplest unit of computation.

#### Process
A process is an isolated computation which consists of one or more threads. The threads within a process can be executed concurrently (simultaneously) and have access to the same resources within the process (memory, executable code and variable values). Processes cannot share these resources with other processes. Each process has its own individual address space, while the threads within that process share their address space.

#### Graph
A way to represent 'things' and their relationships, where a node (circle) is a thing and an edge (line) between nodes is their relationship; e.g. a social network where nodes are people and edges are their friendships.


### Dask

#### Dask architecture
Below is the basic structure of how the components of Dask interact:

![](../images/dask_architecture.png)

Dask uses a pool of workers to process tasks specified by the scheduler. The scheduler determines the tasks to be done by intelligently traversing the task graph. The task graph is produced dynamically and automatically passed to the scheduler when using dask functionality in your code. In the case of dask distributed, the task graph is submitted to the scheduler by the client.

#### Scheduler
One of Dask’s key benefits is its ability to efficiently schedule tasks to optimize computations. Everything Dask does is built on top of “schedulers”. These schedulers take the order of work established by a task graph and find the optimal way to break down and carry out the tasks.

Dask has four types of schedulers:
- Synchronous: Single thread (good for debugging).
        dask.get
- Threaded: Utilises a thread pool.
        dask.threaded.get
- Multiprocessing: Utilises a process pool.
        dask.multiprocessing.get
- Distributed: Utilises a cluster of distributed machines.
        distributed.Client.get

#### Worker
A worker receives tasks to process from the scheduler and returns output to the scheduler when that processing is finished. A worker can be a thread, a process or a whole machine, depending on the scheduler used.

#### Client
A client provides the primary point of access to a distributed scheduler and its associated workers. When using a threaded or multiprocessing scheduler interation with the scheduler is handled by the collection or delayed object.


#### Task Graph
A graph of tasks (nodes) and the data which is required to pass between them (edges).
In the context of dask:
- Task = circle
- Data = box
- Direction of flow = arrow

In [ ]:
def add(a, b):
    return a + b

x = 1
y = 2
z = add(x, y)
w = sum([x, y, z])

![](../images/task_graph_def_img_1.png)

Viewing the relationship between tasks and data can reveal better\* ways to order the tasks to achieve the same computation.

\* Better can mean in less time or using less memory or both!

#### Dask Graph
Dask stores task graphs in a Python dictionary which maps keys to computations:

In [ ]:
dsk = {'x': 1,
       'y': 2,
       'z': (add, 'x', 'y'),
       'w': (sum, ['x', 'y', 'z'])
      }

Dask is different from other parallelising libraries in that it uses ordinary Python structures to represent task graphs instead of a specialised API:

- `{dicts}`
- `(tuples)`
- `functions()`
- `values`


### Exercise: Terminology

Now it is time to test what you have learned! Answer the following questions by assigning each a definition.

Definitions:
1. A group of threads with shared memory.
2. The object which coordinates tasks for workers to process.
3. An interface to access the scheduler in a distrubuted parallel computing cluster.
4. A representation of the relationsips between a set of interdependent tasks and their data.
5. The smallest unit of computation.

Q: What is a scheduler? (Answer in the cell below.)

Q: What is a thread?

Q: What is a task graph?

Q: What is a process?

Q: What is a client?

## Dask Delayed
As users we may want to use the functionality of Dask to parallelise our own code. The `dask.delayed` interface allows us to do this with a very light API:

![](../images/dask_stack.png)

Here we see that all of Dask is built on top of the `scheduler`. The graph spec layer allows graphs to be generated and interpreted by the scheduler. Dask collections such as `arrays`, `bags` and `dataframes` are built on top of these and provide functionality to use these data types without having to think about how to talk to the `scheduler` underneath. Similarly `delayed` provides us with a lightweight interface to create our own parallelisable code without need to directly talk to the scheduler.

The decorator `@delayed` is used to "dask-ify" any arbitrary function, which allows said function to be used as part of a task graph.

In [ ]:
from dask import delayed

@delayed(pure=True)
def add(a, b):
    return a + b

@delayed(pure=True)
def mul(a, b):
    return a * b

@delayed(pure=True)
def inc(a):
    return a + 1

By running the cells below you will see `z` is a delayed object and the dask methods `visualize()` and `compute()` can be used on them like any other dask collection.

In [ ]:
z = add(1, 2)
z

In [ ]:
z.visualize()

In [ ]:
z.compute()

Similarly `c` is a delayed object which can be visualized and computed; it is slightly more complex than `z` on its own.

In [ ]:
a = inc(1)
b = mul(1, 2)
c = add(a, b)
c

In [ ]:
c.visualize()

In [ ]:
c.compute()

It is worth noting that here we have created objects which are "lazy"; when they are created they are not immediately executed. Their execution is `delayed` until a time which we the users (or a scheduler) determine.

### Exercise: 1
Run the code in the cell below and then visualize and compute `total`.

In [ ]:
results = []
for x in range(4):
    a = inc(1)
    b = mul(1, x)
    c = add(a, b)
    results.append(c)

total = delayed(sum, pure=True)(results)
total

**Part 1:** Visualize total.

**Part 2:** Compute total.

## Using Iris Load with Dask
Iris is a Python library for analysing and visualising meteorological and oceanographic data sets. A common issue when using Iris to analyse many data files is that loading these files can take a lot of time when done sequentially. A solution is to parallelise the loading of files, which is an ideal task for Dask.

### Setup

We will need a few more libraries to load files using Iris. These include `iris` for loading files to cubes, `os` and `glob` to access files on disk, and `dask.bag` is the type of dask collection we will use to create a task graph of our parallelised load functions. 

In [ ]:
import iris
import os
import glob
import dask.bag as db

The data we will be loading is monthly northward sea ice velocities from 1890-1899, generated by the Met Office Unified Model and stored in 120 PP files. We will create a list of the filenames, and check the number and names of the files in the list are what we expect, using the following code:

In [ ]:
files = glob.glob(iris.sample_data_path('UM', '*.pp'))

In [ ]:
print('Earliest: {}\nLatest: {}'.format(files[0], files[-1]))

### Load multiple files into multiple cubes using `iris.load_cube`

When using Iris to load data that data is stored in a structure called a `cube`. Multiple cubes can be stored in a list called a `CubeList`. We can load the data from our 120 PP files into a `CubeList` by mapping our list of filenames (`files`) onto the `iris.load_cube()` function:

In [ ]:
cubelist = iris.cube.CubeList(map(iris.load_cube, files))

In [ ]:
len(cubelist)

In [ ]:
type(cubelist)

### Load multiple files into multiple cubes using `dask.bag`

We can parallelise the loading of data from files into cubes by creating a `dask.bag` object which maps our list of filenames onto the function `iris.load_cube()`:

In [ ]:
bag_o_cubes = db.from_sequence(files).map(iris.load_cube)

In [ ]:
type(bag_o_cubes)

In [ ]:
bag_o_cubes.visualize()

`bag_o_cubes` is a delayed (or "lazy") object which we can `visualize` as a task graph. The delayed object will not perform the computation of mapping filenames onto `iris.load_cube()` until we give it the command to do so (namely `bag_o_cubes.compute()`). You can see from the task graph (double-click to enlarge) that each file is being loaded into a cube invidually and in parallel.

Since we want to generate a `CubeList` like the previous example we can compute `bag_o_cubes` from within our `iris.cube.CubeList()` command:

In [ ]:
bag_cubelist = iris.cube.CubeList(bag_o_cubes.compute())

In [ ]:
len(bag_cubelist)

In [ ]:
type(bag_cubelist)

It is worth noting that part of the beauty of Python is that we do not necessarily need to create four objects (`fp`, `files`,  `bag_o_cubes`, `bag_cubelist`) to get a `CubeList` object but can chain the functions together into one big function call:

```python
iris.cube.CubeList(db.from_sequence(files).map(iris.load_cube).compute())
```

### Load multiple files into single cube using `iris.load`

Iris by default will try to merge all input files into the fewest resultant cubes as possible, obeying the rules on what data and metadata can be merged into a single cube. If we know that the result of a load operation will be a single cube we can use `iris.load_cube` to return a single merged cube from the load operation:

In [ ]:
cube = iris.load_cube(files)

print(cube)

Compare this to the output of printing a single cube and it is clear that the decade's worth of data has been merged into a single cube with 120 time steps:

In [ ]:
single_cube = iris.load_cube(files[0])

print(single_cube)

### Load multiple files into single cube using `dask.bag` and `dask.delayed`
We can merge the `CubeList` we generate using a `dask.bag` in exactly the same way:

In [ ]:
bag_merged_cube = bag_cubelist.merge_cube()

print(bag_merged_cube)

However the `merge_cube` step in this does not leverage Dask. We can utilise Dask in the merge step by decorating a call to `merge_cube` with the `@delayed` decorator. This allows us to integrate it with our task graph:

In [ ]:
@delayed
def delayed_merge(cubes):
    return iris.cube.CubeList(cubes).merge_cube()

delayed_merged_cube = delayed_merge(bag_o_cubes)

delayed_merged_cube.visualize()

Once again if you expand the task graph (double-click image) you can see that `merge_cube()` is now part of it. Compare this to the task graph of `bag-o-cubes`. What is different?

`delayed_merged_cube` is "lazy" so we need to call `compute()` to actually execute the task graph and output a merged cube. We can do this within a `print()` call to see that we have merged it successfully:

In [ ]:
print(delayed_merged_cube.compute())

### Exercise: Loading

Consider the following code snippets. Despite looking similar, not all of these code snippets will provide parallel file loading:

In [ ]:
# This cell sets up the files needed by the exercise.
filepath = iris.sample_data_path('GloSea4')
files = glob.glob(filepath)
files

In [ ]:
dlyd = delayed(iris.load_cube)(os.path.join(filepath, '*.pp'))
cs1 = db.from_delayed(dlyd)

In [ ]:
cs2 = db.from_sequence(files).map(iris.load_cube)

In [ ]:
cs3 = db.from_delayed(map(delayed(iris.load_cube), files))

**1.** Print the task graph for each code snippet.

**2.** With reference to the task graph produced by each code snippet, state whether or not each code snippet will provide parallel file loading.

**3.** For the code snippets that do not provide parallel file loading, describe why parallel file loading does not happen and suggest a change to the code snippet to enable parallel file loading.